In [1]:
import heapq

# Definimos el estado inicial y el estado objetivo
estado_inicial = (
    ('M1', '#', '.', 'M3'),
    ('.', '#', '.', '.'),
    ('M2', '.', 'R', '.'),
    ('.', '.', '.', '.')
)

estado_objetivo = (
    ('.', '#', '.', '.'),
    ('.', '#', '.', '.'),
    ('.', '.', '.', '.'),
    ('.', 'M3', 'M2', 'M1')
)

In [2]:
# Función heurística (distancia de Manhattan)
def heuristica(estado_actual, estado_objetivo):
    objetivo_pos = {}
    for i in range(len(estado_objetivo)):
        for j in range(len(estado_objetivo[0])):
            if estado_objetivo[i][j] not in ('.', 'R'):
                objetivo_pos[estado_objetivo[i][j]] = (i, j)

    dist = 0
    for i in range(len(estado_actual)):
        for j in range(len(estado_actual[0])):
            if estado_actual[i][j] not in ('.', '#', 'R'):
                if estado_actual[i][j] in objetivo_pos:
                    dist += abs(i - objetivo_pos[estado_actual[i][j]][0]) + abs(j - objetivo_pos[estado_actual[i][j]][1])
    return dist

In [3]:
# Función para encontrar la ruta óptima usando A*
def a_star(estado_inicial, pos_inicio, pos_final):
    filas = len(estado_inicial)
    columnas = len(estado_inicial[0])
    
    # Define las direcciones posibles de movimiento
    direcciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # Inicializa la cola para A*
    cola = []
    heapq.heappush(cola, (0 + heuristica(estado_inicial, estado_inicial), 0, pos_inicio, estado_inicial, []))
    visitado = set()
    visitado.add((pos_inicio, tuple(map(tuple, estado_inicial))))
    
    # Realiza A*
    while cola:
        _, costo, (x, y), estado_actual, path = heapq.heappop(cola)
        
        if (x, y) == pos_final:
            return path + [(x, y)]
        
        for dx, dy in direcciones:
            nx, ny = x + dx, y + dy
            if 0 <= nx < filas and 0 <= ny < columnas and estado_actual[nx][ny] != '#':
                nuevo_estado = [list(fila) for fila in estado_actual]
                nuevo_estado[nx][ny], nuevo_estado[x][y] = nuevo_estado[x][y], nuevo_estado[nx][ny]
                nuevo_estado_tuple = tuple(map(tuple, nuevo_estado))
                
                if (nx, ny, nuevo_estado_tuple) not in visitado:
                    visitado.add((nx, ny, nuevo_estado_tuple))
                    nuevo_costo = costo + 1
                    prioridad = nuevo_costo + heuristica(nuevo_estado, estado_inicial)
                    heapq.heappush(cola, (prioridad, nuevo_costo, (nx, ny), nuevo_estado_tuple, path + [(nx, ny)]))
    return []

In [4]:
# Función principal para resolver el problema completo
def resolver_problema(estado_inicial, estado_objetivo):
    filas = len(estado_inicial)
    columnas = len(estado_inicial[0])
    
    rutas_completas = []
    acciones = []
    estado_actual = estado_inicial

    # Orden en el que se deben recoger y depositar los objetivos
    orden_objetivos = ['M1', 'M2', 'M3']

    # Encuentra la posición inicial del robot
    for i in range(filas):
        for j in range(columnas):
            if estado_inicial[i][j] == 'R':
                pos_robot = (i, j)
    
    for objetivo in orden_objetivos:
        # Encuentra la posición del objetivo en el estado inicial
        for i in range(filas):
            for j in range(columnas):
                if estado_actual[i][j] == objetivo:
                    pos_objetivo = (i, j)
                    break
        
        # Encuentra la posición del objetivo en el estado objetivo
        for i in range(filas):
            for j in range(columnas):
                if estado_objetivo[i][j] == objetivo:
                    pos_final = (i, j)
                    break
        
        # Encuentra la ruta para mover el robot al objetivo
        ruta = a_star(estado_actual, pos_robot, pos_objetivo)
        rutas_completas.extend(ruta)
        
        for paso in ruta:
            acciones.append(f"mover R fila{paso[0]} columna{paso[1]}")

        # Actualiza el estado del tablero con el objetivo recogido
        estado_actual = cambiar_estado(estado_actual, pos_robot, pos_objetivo, 'R')
        pos_robot = pos_objetivo
        acciones.append(f"cargar R {objetivo} fila{pos_robot[0]} columna{pos_robot[1]}")

        # Encuentra la ruta para mover el objetivo a su posición final
        ruta = a_star(estado_actual, pos_robot, pos_final)
        rutas_completas.extend(ruta)
        
        for paso in ruta:
            acciones.append(f"mover R fila{paso[0]} columna{paso[1]}")
        
        # Actualiza el estado del tablero con el objetivo en su posición final
        estado_actual = cambiar_estado(estado_actual, pos_robot, pos_final, objetivo)
        pos_robot = pos_final
        acciones.append(f"descargar R {objetivo} fila{pos_robot[0]} columna{pos_robot[1]}")
    
    return rutas_completas, acciones

In [5]:
# Función para cambiar el estado del tablero
def cambiar_estado(estado, pos_inicial, pos_final, valor):
    nuevo_estado = [list(fila) for fila in estado]
    nuevo_estado[pos_final[0]][pos_final[1]] = valor
    nuevo_estado[pos_inicial[0]][pos_inicial[1]] = '.'
    return tuple(map(tuple, nuevo_estado))

In [6]:
# Ejecutamos la función y mostramos la ruta completa
ruta_completa, acciones = resolver_problema(estado_inicial, estado_objetivo)
print("Ruta completa:", ruta_completa)
for accion in acciones:
    print(accion)


Ruta completa: [(2, 1), (2, 0), (1, 0), (0, 0), (0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (3, 3), (3, 3), (3, 2), (3, 1), (3, 0), (2, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 2), (2, 2), (1, 2), (1, 3), (0, 3), (0, 3), (0, 2), (1, 2), (2, 2), (2, 1), (3, 1), (3, 1)]
mover R fila2 columna1
mover R fila2 columna0
mover R fila1 columna0
mover R fila0 columna0
mover R fila0 columna0
cargar R M1 fila0 columna0
mover R fila1 columna0
mover R fila2 columna0
mover R fila2 columna1
mover R fila2 columna2
mover R fila2 columna3
mover R fila3 columna3
mover R fila3 columna3
descargar R M1 fila3 columna3
mover R fila3 columna2
mover R fila3 columna1
mover R fila3 columna0
mover R fila2 columna0
mover R fila2 columna0
cargar R M2 fila2 columna0
mover R fila2 columna1
mover R fila2 columna2
mover R fila3 columna2
mover R fila3 columna2
descargar R M2 fila3 columna2
mover R fila2 columna2
mover R fila1 columna2
mover R fila1 columna3
mover R fila0 columna3
mover R fila0 columna3
cargar R M3 fila